# Imports

In [94]:
import sys
sys.path.append('..')
from scripts.extract import convert_xml_to_dataframe
from pathlib import Path

import pandas as pd
import random

In [95]:
random.seed(1) # <- Training
# random.seed(2) # <- Validation

# Read data from xml

In [96]:
df = convert_xml_to_dataframe(path='../data/patients-original.xml')

## Version 2 Generator

In [99]:
# Add boolean for procedures to the dataframe
df['PROCEDURE_A'] = [0]
df['PROCEDURE_B'] = [0]
df['TREATING_PROVIDER_DENTIST'] = [1]
df['TREATING_PROVIDER_FACULTY'] = [0]
df['TREATING_PROVIDER_STUDENT'] = [0]
df = df.drop(['TREATING_PROVIDER'], axis=1)
df = df.drop(0)
df.head()

,PATIENT_ID,SEX,BIRTH_DATE,VISIT_DATE,BLEEDING_ON_PROBING,NR_OF_POCKET,NR_OF_FURCATION,NR_OF_MOBILITY,TOTAL_LOSS_OF_ATTACHMENT_LEVEL,HAS_PARODONTITIS,PROCEDURE_A,PROCEDURE_B,TREATING_PROVIDER_DENTIST,TREATING_PROVIDER_FACULTY,TREATING_PROVIDER_STUDENT


In [103]:
for i in range(10000):
    has_parodontitis = random.randint(0, 2)
    patient_id = 5999977 + i
    treating_provider = random.choice(['dentist', 'faculty', 'student'])

    # Procedures
    procedures_a = 0
    procedures_b = random.randint(0, 1)

    row = {
        'PATIENT_ID': patient_id,
        'SEX': random.choice(['female', 'male']),
        'BIRTH_DATE': '29-02-1960',
        'VISIT_DATE': '15-01-2008',
        'TREATING_PROVIDER_DENTIST': int(treating_provider == 'dentist' if 1 else 0),
        'TREATING_PROVIDER_FACULTY': int(treating_provider == 'faculty' if 1 else 0),
        'TREATING_PROVIDER_STUDENT': int(treating_provider == 'student' if 1 else 0),
        'PROCEDURE_A': procedures_a,
        'PROCEDURE_B': procedures_b,
        'BLEEDING_ON_PROBING': random.randint(0, 25),       # percentage of bleeding (0-100)
        'NR_OF_POCKET': random.randint(0, 1),               # scale 0-4
        'NR_OF_FURCATION': 0,                               # >0 parodontitis, scale 0-5
        'NR_OF_MOBILITY': random.randint(0, 1),
        'TOTAL_LOSS_OF_ATTACHMENT_LEVEL': 0,
        'HAS_PARODONTITIS': 0
    }

    if has_parodontitis == 1:
        row['PROCEDURE_A'] = 1
        row['BLEEDING_ON_PROBING'] = random.randint(20, 100)
        row['NR_OF_POCKET'] = random.randint(1, 4)
        row['NR_OF_FURCATION'] = random.randint(1, 5)
        row['NR_OF_MOBILITY'] = random.randint(2, 3)
        row['TOTAL_LOSS_OF_ATTACHMENT_LEVEL'] = random.randint(1, 3)
        row['HAS_PARODONTITIS'] = 1

    new_record = pd.DataFrame(row, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)

## Check amount of total people having parodontitis

In [104]:
amount_of_people_without_parodontitis = df[df["HAS_PARODONTITIS"] == 0]
amount_of_people_with_parodontitis = df[df["HAS_PARODONTITIS"] == 1]
print(f'People without parodontitis: {len(amount_of_people_without_parodontitis)}')
print(f'People with parodontitis: {len(amount_of_people_with_parodontitis)}')

People without parodontitis: 6638
People with parodontitis: 3372


In [105]:
filepath = Path('../data/patients-v2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)